# Proxy Variable - Violence and Recidivism Comparison

The code bellow compares correlation and network output in an attempt to identitfy Proxy Varaibles.

NETWORK 1

In [1]:
from conx import Network
import random

#seperate into arrays
#for loop on same index in each row
#Feed one network with race and other varaibles
#Feed another network with seriiousness of crime 

def getData(file):
    #open file
    tf = open(file, "r")
    content = tf.readlines()
    row_count = sum(1 for row in content)
    print(row_count)
    #Parse file
    input = []
    content = [x.strip() for x in content[1:]]
    for y in content:
        row = [col for col in y.split(",")]
        #print(row)
        input.append(row)
    tf.close()
    return input

cols = getData("compas-scores-two-years.csv")


7215


In [2]:
#In column 23 there was an issue with splitting the value stored at ","
#Resulting in a list containing 54 columns instead of 53
for c in cols:
    if len(c) == 54:
        c.remove(c[23])

In [3]:
def parseInputs(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            #0
            if r[5] == "Male":
                row.append(1)
            elif r[5] == "Female":
                row.append(0)
            else:
                print("error parsing col 5, Error occurs on row " + str(countWhat))
            #Age: Greater than 45, 25 - 45, and Less than 25
            #1, 2, and 3
            if r[8] == "Greater than 45":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[8] == "25 - 45":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[8] == "Less than 25":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 8, Error occurs on row " + str(countWhat))
            #race = Caucasian, African-American
            if r[9] == "Caucasian":
                row.append(0)
            elif r[9] == "African-American":
                row.append(1)
            else:
                print("error parsing col 9, Error occurs on row " + str(countWhat))
            #Risk of violence Score: Low, Medium, and High
            if r[44] == "Low":
                row.append(0)
            elif r[44] == "Medium":
                row.append(.5)
            elif r[44] == "High":
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #If recidivism 
            if r[52] == '0':
                row.append(0)
            elif r[52] == '1':
                row.append(1)
            else:
                print("error parsing col 52, Error occurs on row " + str(countWhat))
            
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []

            if row != [] and len(row) == 7:
                new.append(row)
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs(cols)  

In [4]:
#Ensure inputs are coorect length
#Input how many rows to be processed
inputs = []
lengthInput = 4000
for r in range(lengthInput):
    inputs.append(tempinputs[r])  
    
inputs = [[x[0:6], x[6:]] for x in inputs]

In [5]:
net = Network(6, 60, 1)

#desired output/target
def des(inputs):
    re = inputs[7]
    return [int(re)]

In [6]:
inputs[1]

[[1, 0, 0, 1, 1, 0], [1]]

In [7]:
net.set_inputs(inputs)

In [8]:
net.reset()
#net.train(max_training_epochs=1000, momentum=1, epsilon=1)
net.train(max_training_epochs=1500, report_rate=250, tolerance = .4)

#stop_percentage: default 1.0, # used in train
#tolerance: default 0.1, # used in train
#report_rate: default 500, # used in train
#epsilon: default 0.1, # learning rate, used in train
#momentum: default 0.9, # used in train
#shuffle=True

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 986.732584858 %correct: 0.24325
Epoch: 250 TSS error: 913.37765676 %correct: 0.4045
Epoch: 500 TSS error: 910.701842975 %correct: 0.425
Epoch: 750 TSS error: 909.071680722 %correct: 0.41525
Epoch: 1000 TSS error: 909.291139453 %correct: 0.42325
Epoch: 1250 TSS error: 905.054108213 %correct: 0.428
Epoch: 1500 TSS error: 908.571898632 %correct: 0.41025
--------------------------------------------------
Epoch: 1500 TSS error: 908.571898632 %correct: 0.41025


In [9]:
net.test()

--------------------------------------------------
Test:
******************************
Input : [1, 0, 1, 0, 1, 0.5]
Output: [ 0.66514002] 
Target: [1] Correct
******************************
Input : [1, 0, 1, 0, 0, 0.5]
Output: [ 0.64315229] 
Target: [0] Incorrect
******************************
Input : [1, 0, 1, 0, 1, 0]
Output: [ 0.42643226] 
Target: [0] Incorrect
******************************
Input : [1, 0, 0, 1, 1, 0.5]
Output: [ 0.55019504] 
Target: [0] Incorrect
******************************
Input : [1, 1, 0, 0, 0, 0]
Output: [ 0.24048882] 
Target: [0] Correct
******************************
Input : [1, 0, 1, 0, 1, 0]
Output: [ 0.42643226] 
Target: [0] Incorrect
******************************
Input : [1, 0, 1, 0, 0, 0]
Output: [ 0.37479175] 
Target: [1] Incorrect
******************************
Input : [0, 0, 0, 1, 1, 0.5]
Output: [ 0.45004431] 
Target: [0] Incorrect
******************************
Input : [1, 0, 1, 0, 0, 1]
Output: [ 0.73971389] 
Target: [1] Correct
**************

In [12]:
#try network on test data
results = []
forit = len(tempinputs) - 4000
for h in range(forit):
    val = tempinputs[h + 4000]
    val = val[0:6]
    results.append(net.propagate(val))

NETWORK 2

In [13]:
def parseInputs2(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            #0
            if r[5] == "Male":
                row.append(1)
            elif r[5] == "Female":
                row.append(0)
            else:
                print("error parsing col 5, Error occurs on row " + str(countWhat))
            #Age: Greater than 45, 25 - 45, and Less than 25
            #1, 2, and 3
            if r[8] == "Greater than 45":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[8] == "25 - 45":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[8] == "Less than 25":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 8, Error occurs on row " + str(countWhat))
            #race = Caucasian, African-American
            if r[9] == "Caucasian":
                row.append(0)
            elif r[9] == "African-American":
                row.append(1)
            else:
                print("error parsing col 9, Error occurs on row " + str(countWhat))
            #Risk of violence Score: Low, Medium, and High
            if r[40] == "Low":
                row.append(0)
            elif r[40] == "Medium":
                row.append(.5)
            elif r[40] == "High":
                row.append(1)
            else:
                print("error parsing col 40, Error occurs on row " + str(countWhat))
            #If recidivism 
            if r[52] == '0':
                row.append(0)
            elif r[52] == '1':
                row.append(1)
            else:
                print("error parsing col 52, Error occurs on row " + str(countWhat))
            
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []

            if row != [] and len(row) == 7:
                new.append(row)
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs2(cols)  

error parsing col 40, Error occurs on row 4754


In [14]:
#Reset inputs
inputs = []
for r in range(lengthInput):
    inputs.append(tempinputs[r])  
    
inputs = [[x[0:6], x[6:]] for x in inputs]

In [15]:
net2 = Network(6, 60, 1)

#desired output/target
def des(inputs):
    re = inputs[7]
    return [int(re)]

In [16]:
inputs[0]

[[1, 0, 1, 0, 1, 0], [1]]

In [17]:
net2.set_inputs(inputs)

In [19]:
net2.reset()
#net.train(max_training_epochs=1000, momentum=1, epsilon=1)
net2.train(max_training_epochs=1500, report_rate=250, tolerance = .4)

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 2046.18579288 %correct: 0.46325
Epoch: 250 TSS error: 865.812949766 %correct: 0.51
Epoch: 500 TSS error: 863.810129838 %correct: 0.502
Epoch: 750 TSS error: 862.635209558 %correct: 0.508
Epoch: 1000 TSS error: 864.148725091 %correct: 0.50625
Epoch: 1250 TSS error: 863.319997271 %correct: 0.5125
Epoch: 1500 TSS error: 864.377594277 %correct: 0.50725
--------------------------------------------------
Epoch: 1500 TSS error: 864.377594277 %correct: 0.50725


In [20]:
net2.test()

--------------------------------------------------
Test:
******************************
Input : [0, 0, 1, 0, 0, 0]
Output: [ 0.29145051] 
Target: [1] Incorrect
******************************
Input : [1, 0, 1, 0, 0, 0]
Output: [ 0.34334247] 
Target: [0] Correct
******************************
Input : [1, 0, 1, 0, 1, 1]
Output: [ 0.73751657] 
Target: [1] Correct
******************************
Input : [0, 0, 1, 0, 0, 0]
Output: [ 0.29145051] 
Target: [0] Correct
******************************
Input : [1, 1, 0, 0, 0, 0]
Output: [ 0.22112524] 
Target: [1] Incorrect
******************************
Input : [1, 0, 0, 1, 0, 0]
Output: [ 0.37058419] 
Target: [0] Correct
******************************
Input : [1, 0, 1, 0, 0, 0]
Output: [ 0.34334247] 
Target: [0] Correct
******************************
Input : [0, 0, 1, 0, 1, 0]
Output: [ 0.23260347] 
Target: [0] Correct
******************************
Input : [1, 0, 1, 0, 0, 0]
Output: [ 0.34334247] 
Target: [0] Correct
******************************

Look at data range = len(tempinputs) - 4000

In [28]:
results2 = []
forIt = (len(tempinputs) - 4000)
for p in range(forIt):
    val = tempinputs[p + 4000]
    val = val[0:6]
    results2.append(net2.propagate(val))
    

In [29]:
print("Proxy Precentage: ")  

Proxy Precentage: 


In [30]:
countSame = 0
for s in range(len(results2)):
    if (results[s] > .5 and results2[s] > .5) or (results[s] < .5 and results2[s] < .5):
        countSame += 1
        
print(countSame)
print(len(results))
print("Proxy Precentage: ")      
print(float(countSame)/float(len(results)))

1066
2150
Proxy Precentage: 
0.4958139534883721


In [31]:
def getProxy(results, results2):
    countSame = 0
    for s in range(len(results2)):
        bothTrue = results[s] > .5 and results2[s] > .5
        bothFalse = results[s] < .5 and results2[s] < .5
        if bothTrue or bothFalse:
            countSame += 1     
    return (float(countSame)/float(len(results)))

print(getProxy(results, results2))

0.4958139534883721


In [ ]:
#5 in net1
#5 in net1

res2 = []
countSame = 0
forIt = (len(tempinputs) - 4000)
for p in range(forIt):
    val = tempinputs[p + 4000]
    res2.append(val[5])